In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd ./drive/My\ Drive/

/content/drive/My Drive


In [0]:
%tensorflow_version 1.4
import os
import argparse
import model_training.sentiment_dataset as sentiment_dataset
import model_training.sentiment_model_cnn as sentiment_model_cnn
import model_training.config_holder as config_holder
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.4`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import GlobalMaxPool1D
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

In [0]:
args = {'config_file': './model_training/training_config.json',
       'num_epoch': 3,
       'train': './model_training/data/train/',
       'validation': './model_training/data/dev/',
       'eval': './model_training/data/eval/'
       }

In [0]:
print("Preparing for training...")

training_config = config_holder.ConfigHolder(args['config_file']).config

training_config["num_epoch"] = args['num_epoch']

train_dataset = sentiment_dataset.train_input_fn(args['train'], training_config)
validation_dataset = sentiment_dataset.validation_input_fn(args['validation'], training_config)
eval_dataset = sentiment_dataset.eval_input_fn(args['eval'], training_config)

#train: 1,360,000
#dev: 160,000
#eval: 80,000


Preparing for training...
Fetching train data...
train
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
Fetching validation data...
dev
Fetching eval data...
eval


In [0]:
embeddings_index = dict()
f = open('model_training/dictionary/glove.twitter.27B.200d.txt', encoding="utf-8")
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.array(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1193515 word vectors.


In [0]:
vocab_size = len(embeddings_index.keys())

In [0]:
n = len(embeddings_index.keys())
m = len(embeddings_index['the'])

embedding_matrix = np.zeros((n,m))
for index, key in zip(range(0, n), embeddings_index.keys()):
    embedding_matrix[index] = embeddings_index[key]

In [0]:
embedding_matrix.shape

(1193515, 200)

##Benchmark

In [0]:
# define model
model = Sequential()

# Layer 1: Embedding layer
# This layer should load the embeddings vectors from your dictionary as a numpy array
# - input_leght should be equal to your padding length
# - input_dim should be the length of your word list
# - output_dim should be the size your your embedding vectors
# - trainable True
model.add(Embedding(vocab_size, 200, weights=[embedding_matrix], input_length=100, trainable=True, name='embedding'))

# Layer 2: Convolution1D layer
# - filters 100
# - kernel_size 2
# - strides 1
# - padding 'valid'
# - activation 'relu'
model.add(Conv1D(filters = 100, kernel_size = 2, strides = 1, padding = 'valid', activation = 'relu'))

# Layer 3: GLobalMaxPool1D layer
model.add(GlobalMaxPool1D())

# Layer 4: Dense layer
# - units 100
# - activation 'relu'
model.add(Dense(100, activation = 'relu'))

# Layer 5: Dense layer
# - units 1
# - activation 'sigmoid'
model.add(Dense(1, activation = 'sigmoid'))

adam = Adam(lr=0.0005)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 200)          238703000 
_________________________________________________________________
conv1d (Conv1D)              (None, 99, 100)           40100     
_________________________________________________________________
global_max_pooling1d (Global (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 238,753,301
Trainable params: 238,753,301
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(x = train_dataset[0], 
          y = train_dataset[1], 
          steps_per_epoch = train_dataset[2]["num_batches"],
          epochs = training_config["num_epoch"],
          validation_data = (validation_dataset[0]['embedding_input'], validation_dataset[1]),
          validation_steps = validation_dataset[2]["num_batches"])

score = model.evaluate(
eval_dataset[0], eval_dataset[1], steps=eval_dataset[2]["num_batches"], verbose=0)

print("Test loss:{}".format(score[0]))
print("Test accuracy:{}".format(score[1]))

Train on 1360 samples, validate on 1000 samples
Epoch 1/3
1360/1360 [==============================] - 346s 255ms/step - loss: 0.4635 - acc: 0.7766 - val_loss: 0.4426 - val_acc: 0.7911
Epoch 2/3
1360/1360 [==============================] - 324s 238ms/step - loss: 0.4221 - acc: 0.8033 - val_loss: 0.4378 - val_acc: 0.7944
Epoch 3/3
1360/1360 [==============================] - 323s 238ms/step - loss: 0.3966 - acc: 0.8186 - val_loss: 0.4399 - val_acc: 0.7941
Test loss:0.4395864550024271
Test accuracy:0.7940875291824341


In [0]:
score = model.evaluate(
eval_dataset[0], eval_dataset[1], steps=eval_dataset[2]["num_batches"], verbose=0)

print("Test loss:{}".format(score[0]))
print("Test accuracy:{}".format(score[1]))

Test loss:0.4395864550024271
Test accuracy:0.7940875291824341


In [0]:
folder = 'output_model_benchmark'
output = os.path.join(folder, "sentiment_model.h5")
tf.saved_model.save(model, os.path.join(output, "1"))

INFO:tensorflow:Assets written to: output_model_benchmark/sentiment_model.h5/1/assets
